In [132]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 300
pd.options.display.max_columns = 999

In [133]:
#Read data
df = pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0)

In [134]:
#Save DQs before removing any information
dq_indexes = df[df.apply(lambda r: r.str.contains('(BMB|None|DQ|Disq)', case=False).any(), axis=1)].index

C:\Users\Joe\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [135]:
#Inspect data
df.head()

,CompID,Name,Sex,Class,Division,BestBP,Total,Place,BestSQ,BestDL,BDW,BP1,BP2,BP3,Wilks,SQ1,SQ2,SQ3,DL1,DL2,DL3,DOB
Index,,,,,,,,,,,,,,,,,,,,,,
0,1,Barbara Hamilton,F,56,O/50,42.5/43,42.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Nat Zacharko,F,56,O/40,60,60,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Dean Jenkinson,M,67.5,Open,125,125,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron McFarlane,M,67.5,Open,105,105,2-Jan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,David Graham,M,67.5,O/60,52.5,52.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Goals:
- Homogenize column values
- Fix data types
- Change no-lifts to minus (rather than x suffix)

##### Sex column

In [136]:
#Inspect
df.Sex.value_counts()

M           15488
F            6969
M-O            63
M-J            43
m              42
F-O            18
F-J            17
M-M1           16
f              16
M - Open       12
F-M1           11
M-SJ           11
M-M3           11
F - M1          7
F - Open        6
M-M2            6
M - O           6
M-Sub           5
W               4
M - M2          3
F-SJ            2
F - M2          2
M-BL            2
F-M3            2
M - U18         1
M - M1          1
F-M2            1
M - SJ          1
M-Jnr           1
F-M4            1
Name: Sex, dtype: int64

In [137]:
#Fix
def fix_sex(df):
    df.Sex = df.Sex.replace({'m': 'M', 'f': 'F', 'W': 'F'})
    df.Sex = df.Sex.str.replace(' ','')
    df[df.Sex.str.contains('-')].Division.isnull().all() #All rows with div info in sex column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Division'], inplace = True)
    df.Sex = pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Sex']
    return df

##### Class column

In [138]:
#Inspect
#df[~df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Possibly bad values
df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Okay values
#df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts().plot(kind='bar') #Histogram of okay values

82.5      846
90        767
75        735
100       598
67.5      504
83        483
93        473
110       459
105       447
125       390
74        315
120       261
125+      238
60        228
56        173
120+      167
66        166
52        162
63        145
72        121
57        110
84         91
84+        90
90+        85
48         56
59         42
47         37
44          5
89.2        1
92.54       1
120.8       1
102         1
66.9        1
75.5        1
62.5        1
117.26      1
89.6        1
67.6        1
76.4        1
55.9        1
Name: Class, dtype: int64

In [139]:
#Fix
def fix_class(df):
    df.Class = df.Class.str.replace('kg', '')
    df.Class = df.Class.str.replace('x', '')
    df.Class = df.Class.str.replace(' ', '')
    df[df.Class.fillna('').str.contains('-')].Division.isnull().all() #All rows with div info in class column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Division'], inplace = True)
    df.Class = pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Class']
    df.Class.replace('', np.nan, inplace = True)
    #Look up special case comp tables to check for errors
    df.replace({'Class' : { '125.00+' : '125+', '66.9': '67.5', '55.9': '56', '62.5': '67.5', '67.6': '67.5', '75.5': '75'}}, inplace = True)
    for i in [1338, 1556, 1558, 3446, 3473, 7510, 7528]:
        temp = df.loc[i, 'BDW']
        df.loc[i, 'BDW'] = df.loc[i, 'Class']
        df.loc[i, 'Class'] = temp
    return df

##### DOB column

In [211]:
#Inspect
df.DOB.value_counts()
#set([k[0] for j, k in df[df.DOB.notnull()].iterrows() if any(substring in k[21] for substring in ['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'])])

1993.0    955
1994.0    827
1992.0    827
1995.0    782
1991.0    642
1989.0    636
1990.0    627
1996.0    623
1997.0    548
1987.0    505
1988.0    479
1986.0    373
1998.0    359
1982.0    357
1985.0    325
1984.0    310
1981.0    299
1983.0    281
1980.0    261
1999.0    237
2000.0    206
1979.0    205
1977.0    190
1975.0    169
1978.0    157
1973.0    152
1970.0    150
1974.0    145
2001.0    137
1976.0    129
2002.0    124
1969.0    123
1965.0    117
1966.0    113
1971.0    106
1968.0    103
1967.0    102
1972.0    101
1964.0     95
1961.0     91
1962.0     86
1956.0     79
1948.0     74
1963.0     67
1958.0     63
1960.0     58
2003.0     52
1959.0     49
1957.0     42
1955.0     41
1949.0     37
2004.0     35
1945.0     32
1954.0     31
1947.0     29
1950.0     29
1953.0     28
1952.0     26
1944.0     20
1951.0     16
1939.0     15
1941.0     11
1943.0      7
2005.0      7
1940.0      6
1938.0      2
2006.0      2
1932.0      2
1946.0      1
1933.0      1
1930.0      1
2007.0

In [141]:
#Fix
def fix_dob(df):
    #Division in DOB column
    for i in [103, 109, 113, 114, 141, 142, 144, 146, 168, 177, 184, 214]:
        df.loc[df.CompID == i, 'Division'] = df.loc[df.CompID == i, 'DOB']
        df.loc[df.CompID == i, 'DOB'] = np.nan
    #Words in DOB column
    df.DOB.replace(['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'], np.nan, inplace = True)
    df.DOB.replace('`987', 1987, inplace = True)
    #Big numbers
    df.DOB.replace('\d{5}', np.nan, inplace = True, regex = True)
    #Small numbers
    df.replace({'DOB' : { '0' : np.nan, '1886': 1986, '1680': 1980, '995': 1995}}, inplace = True)
    df.loc[17261, 'DOB'] = 2018-30; df.loc[12972, 'DOB'] = 2016-35; df.loc[12975, 'DOB'] = 2018-30; df.loc[18635, 'DOB'] = 2018-27
    for i,j in zip([572, 574, 606, 612, 665, 939, 940], [2015, 2015, 2016, 2016, 2017, 2020, 2020]):
        df.loc[df.CompID == i, 'DOB'] = j-df.loc[df.CompID == i, 'DOB']
    #Too close to current year
    df.replace({'DOB' : { 2014 : np.nan, 2016: np.nan, 2020: np.nan}}, inplace = True)
    df.loc[12166, 'DOB'] = np.nan
    return df

##### Division column

In [142]:
#Inspect
df.Division.value_counts()
#Check rows with weird entries
#set([k[0] for j, k in df[df.Division.notnull()].iterrows() if any(substring in k[4] for substring in ['"'])])

O              1887
Open            681
J               503
M1              498
M2              310
SJ              212
M3              192
O/40            160
OPEN            131
"               114
O/50             90
O/60             89
U/23             80
Jnr              54
M4               50
VIC              35
Op               31
JNR              30
U18              26
o                23
U/19             21
Jun              20
BP               20
Classic          20
MI               18
Junior           17
O-O              17
Bp               17
U23              16
open             15
Nov              15
SJNR             15
SUB JNR          13
O-O/40           12
U/18             11
M                10
JR                9
0/50              9
MII               8
O/70              8
Guest             8
b                 8
0/40              7
Uni               7
OPEN?             7
MIII              7
OVER 40           7
Oceania           6
Op-O/40           6
APFC              6


In [143]:
#Fix
def fix_division(df):
    df.loc[df.CompID == 64, 'Division'] = df[df.CompID == 64].Division.str.replace('"', 'Snr')
    df.Division = df.Division.str.replace('"', 'Open')
    divsemdf = pd.read_csv("../../Data/Division_semantics.csv", engine = 'python')
    #Map words to semantic meaning
    semdict = {tuple(divsemdf[divsemdf[col].notnull()][col]): col for col in divsemdf.columns}
    semmap = {}
    for k, v in semdict.items():
        for key in k:
            semmap[key] = v
    #Change words
    df.Division.replace(semmap, inplace = True)
    df.Division.replace('Unknown', np.nan, inplace = True)
    df.loc[[5546, 3277], 'Division'] = 'MI'
    return df

##### Wilks column

In [144]:
#Inspect
#df.Wilks.value_counts()
df[~df.Wilks.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Wilks.value_counts()
#df[df.Wilks.str.match('\.$') == True] #Check none start with .
#df[df.Wilks.str.match('^\.') == True] #Check none end with .

BMB       396
-         371
x           6
out         5
Disq.       4
         ... 
55,71       1
396,78      1
112,67      1
103,97      1
268,82      1
Name: Wilks, Length: 647, dtype: int64

In [145]:
#Fix
def fix_wilks(df):
    df.Wilks.replace(['0', 'BMB', '-', 'out', 'x', 'Disq.', 'Disq', '?', 'DQ (fees)', 'withdrawn', 'DQ'], np.nan, inplace = True)
    df.Wilks = df.Wilks.str.replace(',', '.', regex = False)
    return df

##### BDW column

In [146]:
#Inspect
#df.BDW.value_counts()
df[~df.BDW.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].BDW.value_counts()

81,85     5
92,20     5
73,85     5
73,80     4
55,00     4
         ..
121,60    1
82,80     1
76,00     1
61,15     1
96,10     1
Name: BDW, Length: 490, dtype: int64

In [147]:
#Fix
def fix_bdw(df):
    df.BDW = df.BDW.str.replace(',', '.', regex = False)
    df.BDW = df.BDW.str.replace('..', '.', regex = False)
    df.BDW = df.BDW.str.replace('?', '', regex = False)
    df.BDW.replace('BMB', np.nan, inplace = True)
    return df

##### Place column

In [148]:
#Inspect
df.Place.value_counts()
#df[df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Fine
#df[~df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Bad

#Bad categories:
#df[~df.Place.fillna('').str.contains('\d')].Place.value_counts() #No info
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.value_counts() #Suffix + Additional info included
#df[df.Place.fillna('').str.match('.+(kg$)')] #All of these have division/class info already - can remove from place column
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)$')].Place.value_counts() #Has "th", "st", "nd", "rd" suffix
#df[df.Place.fillna('').str.contains('[a-zA-Z]')].Place.value_counts() #Additional info (from one comp only)
#df[~df.Place.fillna('1').str.isnumeric()].CompID.unique() #Comps with placing for 2 classes

1                7661
2                3571
3                2033
4                1209
5                 726
6                 459
7                 259
8                 172
-                 148
9                 111
10                 74
1st                61
11                 58
?-?                47
12                 44
GOLD               43
BMB                43
2nd                42
-?                 38
13                 30
1 & 1              30
3rd                28
4th                20
14                 20
SILVER             17
15                 16
5th                15
BRONZE             11
16                 11
7th                10
18                  8
x                   8
17                  8
19                  8
6th                 8
9th                 7
12th                7
10th                6
20                  6
1.1                 5
22                  5
21                  5
1st M1 120+kg       4
DQ                  4
1 & 3               4
1st M1 120

In [149]:
#Fix
def fix_place(df):
    df.Place.replace(['0', '-', '?-?', '-?', 'BMB', 'x', 'DQ', 'Disq.', '?', 'Inv', '?-??', 'X', 'Guest', 'N.E.', 'Most Imp', 'G', 'Disq', 'bmb'], np.nan, inplace = True)
    df.replace({'Place' : {'GOLD': '1', 'SILVER': '2', 'BRONZE': '3', '`.1': '1', '`1': '1'}}, inplace = True)
    df.Place = df.Place.str.replace('Jan', '1', regex = False)
    df.Place = df.Place.str.replace('Feb', '2', regex = False)
    df.Place = df.Place.str.replace('Mar', '3', regex = False)
    df.Place = df.Place.str.replace('-Nov', '', regex = False)
    df.Place = df.Place.str.replace('?', ' ', regex = False)
    df.loc[df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].index, 'Place'] = df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.str.split(' ', expand = True)[0].str.strip('stnrdh') #Terrible
    df.loc[df[df.Place.fillna('').str.contains('[a-zA-Z]')].index, 'Place'] = '1'
    df.loc[df[~df.Place.fillna('1').str.isnumeric()].index, 'Place'] = np.nan
    return df

##### Name column

In [150]:
#Inspect
#df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name #Names that have characters other than a-z, spaces, apostrophe and dash.
#df[df.Name.str.contains("\s$")].Name
#df[df.Name.str.contains("- ")].Name

In [151]:
#Fix
def fix_name(df):
    df.loc[12467, 'Name'] = 'Steven Melekiola'
    df.loc[2036, 'Name'] = "MAX O'TOOLE"
    df.loc[1491, 'Name'] = "Mark Shillington"
    df.loc[2706, 'Name'] = "MARK O'SHEA"
    df.loc[7688, 'Name'] = "Clair Angel"
    df.loc[3088, 'Name'] = "Rick Lewis"
    df.Name = df.Name.str.replace('?', '', regex = False)
    df.Name = df.Name.str.replace("'Sasha'", '', regex = False)
    df.Name = df.Name.str.replace('*', '', regex = False)
    df.Name = df.Name.str.replace('- ', '-', regex = False)
    df.Name = df.Name.str.replace(' -', '-', regex = False)
    df.Name = df.Name.str.replace('4th BP', '', regex = False)
    df.Name = df.Name.str.replace(' - u18', '', regex = False)
    df.Name = df.Name.str.replace('u18', '', regex = False)
    df.Name = df.Name.str.replace('??QLD', '', regex = False)
    df.Name = df.Name.str.replace('QLD', '', regex = False)
    df.Name = df.Name.str.replace(r'\([^)]*\)', '')
    df.Name = df.Name.str.replace(r'\"[^)]*\"', '')
    df.loc[df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].index, 'Name'] = df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name.str.split(', ').str[::-1].str.join(' ')
    df.Name = df.Name.str.replace('(\s{2})', '')
    df.Name = df.Name.str.strip()
    return df

##### Total column

In [152]:
#Inspect
#df.Total.value_counts() #1380
#df[df.Total.fillna('').str.contains("[a-zA-Z]")].Total.value_counts()
df[df.Total.fillna('').str.match('^\d+\.?\d?$')].Total.value_counts()

0        229
600      184
500      179
520      167
570      138
        ... 
320.5      1
133        1
937.5      1
555.5      1
399        1
Name: Total, Length: 1117, dtype: int64

In [153]:
#Fix
def fix_total(df):
    df.Total.replace(['-', '0,0', '0', 'Out', 'Disq.', 'withdrawn', 'DSQ', 'Disq', 'BMB', 'BMB?', 'bmb', 'x', 'X', 'out'], np.nan, inplace = True)
    df.Total = df.Total.str.replace('?', '', regex = False)
    df.loc[df[df.Total.fillna('').str.contains("[a-zA-Z]")].index, 'Total'] = df[df.Total.fillna('').str.contains("[a-zA-Z]")].Total.str.split(' ', expand = True)[0].str.strip('RANWMY')
    df.Total = df.Total.str.replace(',', '.', regex = False)
    return df

##### Best columns

In [154]:
#Inspect
#df.BestSQ.value_counts()
#df.BestBP.value_counts()
df.BestDL.value_counts()
#df[df.BestSQ.fillna('').str.contains("x|X")].BestSQ.value_counts()
#df[df.BestBP.fillna('').str.contains("x|X")].BestBP.value_counts()
#df[df.BestDL.fillna('').str.contains("x|X")].BestDL.value_counts()
#df.BestSQ.str.strip('WNR ').value_counts()
#df.BestBP.str.strip('kgBMBWNR ').value_counts()
#df.BestDL.str.strip('WNRF ').value_counts()

-              80
240            59
220            56
230            54
200            53
250            41
210            39
190            38
185            35
235            34
180            33
160            32
225            30
175            25
140            25
170            24
245            24
150            23
260            23
120            23
130            22
205            22
215            21
155            21
280            20
270            19
255            17
100            16
115            15
125            15
242.5          13
195            13
165            13
145            12
x              12
265            11
95             11
252.5          11
142.5          11
135            11
222.5          10
207.5          10
90             10
110             9
0               9
65              9
92.5            9
275             9
237.5           9
285             9
127.5           8
227.5           8
132.5           8
320             7
162.5           7
182.5     

In [155]:
#Fix
def fix_bests(df):
    df.BestSQ.replace(['-', 'BMB', '0', '?', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.BestBP.replace(['-', 'x', 'BMB', '0', '?','260 BMB', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.BestDL.replace(['-', 'BMB', '0', '?', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.loc[df[df.BestSQ.fillna('').str.contains("x|X")].index, 'BestSQ'] = np.nan
    df.loc[df[df.BestBP.fillna('').str.contains("x|X")].index, 'BestBP'] = np.nan
    df.loc[df[df.BestDL.fillna('').str.contains("x|X")].index, 'BestDL'] = np.nan
    df.BestSQ = df.BestSQ.str.strip('WNR ')
    df.BestBP = df.BestBP.str.strip('kgBMBWNR ')
    df.BestDL = df.BestDL.str.strip('WNRF ')
    df.BestBP = df.BestBP.str.replace('/', ' ', regex = False)
    df.BestBP = df.BestBP.str.replace('o', '0', regex = False)
    df.BestDL = df.BestDL.str.replace('/', ' ', regex = False)
    df.loc[1862, 'BestDL'] = np.nan
    df.loc[df[df.BestBP.fillna('').str.contains(" ")].index, 'BestBP'] = df[df.BestBP.fillna('').str.contains(" ")].BestBP.str.split(' ', expand = True)[0]
    df.loc[df[df.BestDL.fillna('').str.contains(" ")].index, 'BestDL'] = df[df.BestDL.fillna('').str.contains(" ")].BestDL.str.split(' ', expand = True)[0]    
    return df

##### Attempt columns

In [156]:
#Inspect
col = 'DL3'
#df[col].value_counts().nlargest(100)
#df[col].value_counts().nsmallest(100)
#df[df[col].fillna('').str.match('.*[xXfF]$')][col].value_counts() #XxFf at the end
#df[df[col].fillna('').str.match('^[a-zA-Z]+$')][col].value_counts() #Made of letters
#df[df[col].fillna('').str.contains(',')][col].unique() #Contains comma as decimal
#df[~df[col].fillna('').str.match('^-?\d+\.?\d?$')][col].value_counts() #Not of the form ddd.ddd 
alls = []
for col in [i + j for i in ['SQ', 'BP', 'DL'] for j in ['1', '2', '3']]:
    alls.extend(df[~df[col].fillna('').str.match('[0-9.-]*')][col].unique())
set(alls)

set()

In [177]:
#Fix
def general_fix(df, col):
    df[col].replace(['8', '-', 'x', 'X', 'BMB', 'out', 'F', 'PASS', '??X', '150-x', '0,0', 'x?', '0', '1', '##',
                    '4th at', '?''??????n', 'BP & DL ONLY', 'Push Pull only', 'Withdrawn Injured', '_amp_nbsp_semi_', '`'], np.nan, inplace = True)
    df[col] = df[col].str.replace(',', '.', regex = False)
    df[col] = df[col].str.replace('*', '', regex = False)
    df[col] = df[col].str.replace('`', '', regex = False)
    df[col] = df[col].str.replace('?', '', regex = False)
    df[col] = df[col].str.replace('-or', '', regex = False)
    df[col] = df[col].str.replace('n', '', regex = False)
    df.loc[df[df[col].fillna('').str.match('.*\s?[xXfF]$')].index, col] = '-' + df[df[col].fillna('').str.contains('.*\s?[xXfF]$')][col].str.strip('xfXF ')
    df.loc[df[df[col].fillna('').str.match('^x\s?.*')].index, col] = '-' + df[df[col].fillna('').str.contains('^x\s?.*')][col].str.strip('x ')
    df[col].replace('', np.nan, inplace = True)
    df.loc[df[df[col].fillna('').str.contains(" ")].index, col] = df[df[col].fillna('').str.contains(" ")][col].str.split(' ', expand = True)[0]
    df.loc[df[df[col].fillna('').str.contains("[a-zA-Z]")].index, col] = df[df[col].fillna('').str.contains("[a-zA-Z]")][col].str.strip('ARSW')
    return df

def fix_attempts(df):
    df.loc[2491, 'BP2'] = '202.5'
    df.loc[22327, 'SQ3'] = '140'
    df.loc[1277, 'DL3'] = '200F'
    df.loc[8692, 'DL2'] = '235x'
    df.loc[2924, 'BP3'] = '107.5x'
    df.loc[4420, 'DL2'] = '157.5'
    for col in [i + j for i in ['SQ', 'BP', 'DL'] for j in ['1', '2', '3']]:
        df = general_fix(df, col)
    return df

##### Fix errors

In [158]:
#Inspect
#df[df.SQ3 == '1470']
#df[df.BDW == '1090.9']
#df[df.BDW == '1009.7']
#df[df.BDW == '998.6']
#df[df.BP3 == '-1075']
#df[df.DL2 == '-2235']
#df[df.DL3 == '-00']
#df[df.Wilks.astype(float) < 10]
#df[df.BDW == '0']
#df[df['DL2'].fillna('').str.match('.*\.\d{2,}$')]
#df[df['BP2'].fillna('').str.match('.*\.\d{2,}$')]

#Check comps with no failures (failure possibly indicated by strikethrough in source)
#df.groupby('CompID').filter(lambda x: (x[['SQ1','SQ2','SQ3', 'BestSQ','BP1','BP2','BP3','BestBP','DL1','DL2','DL3','BestDL']].fillna(1) > 0).all(axis=1).all()).CompID.unique()

In [178]:
#Fix
def fix_errors(df):
    df.loc[750, 'BDW'] = '109.90'
    df.loc[659, 'BDW'] = '109.7'
    df.loc[1512, 'BDW'] = '98.6'
    df.loc[22327, 'Wilks'] = '365'
    df.loc[df[df.Wilks.astype(float) < 10].index, 'Wilks'] = np.nan
    df.loc[df[df.BDW.astype(float) < 10].index, 'BDW'] = np.nan
    return df

##### Interpolation

##### Data organization

In [215]:
def restructure(df):
    df = df[['CompID','Name','Sex','DOB','BDW','Class','Division','SQ1','SQ2','SQ3', 'BestSQ','BP1','BP2','BP3','BestBP','DL1','DL2','DL3','BestDL','Total','Wilks','Place']]
    df = df.astype({'CompID': 'int64', 'Name': 'string', 'Sex': 'string', 'DOB': 'float64', 'BDW': 'float64', 'Class': 'string', 'Division': 'string', 'SQ1': 'float64', 'SQ2': 'float64', 'SQ3': 'float64', 
                    'BestSQ': 'float64', 'BP1': 'float64', 'BP2': 'float64', 'BP3': 'float64', 'BestBP': 'float64', 'DL1': 'float64', 'DL2': 'float64', 'DL3': 'float64', 'BestDL': 'float64', 
                    'Total': 'float64', 'Wilks': 'float64', 'Place': 'float64'})
    return df

### Execute

In [213]:
#One function
def fix_all(df):
    df = fix_sex(df)
    df = fix_class(df)
    df = fix_dob(df)
    df = fix_division(df)
    df = fix_wilks(df)
    df = fix_bdw(df)
    df = fix_place(df)
    df = fix_name(df)
    df = fix_total(df)
    df = fix_bests(df)
    df = fix_attempts(df)
    df = fix_errors(df)
    df = restructure(df)
    return df

def interpolate_all(df):
    return df

df = fix_all(pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0))
df = interpolate_all(df)

In [180]:
df.to_csv("../../Data/Clean_Full.csv", index_label = 'Index')

# Testing area

In [216]:
df.isnull().sum()/len(df)*100

CompID       0.000000
Name         0.000000
Sex          0.000000
DOB         38.870344
BDW          4.001230
Class       63.598032
Division    73.533029
SQ1         21.560963
SQ2         21.947470
SQ3         23.137737
BestSQ      94.364898
BP1          9.113668
BP2          9.548489
BP3         11.125264
BestBP      92.089775
DL1         21.266690
DL2         21.907941
DL3         23.317814
BestDL      94.255095
Total        6.592586
Wilks       11.353654
Place       25.017569
dtype: float64

In [266]:
df.groupby('CompID').filter(lambda x: (x[['SQ1','SQ2','SQ3', 'BestSQ','BP1','BP2','BP3','BestBP','DL1','DL2','DL3','BestDL']].fillna(1) > 0).all(axis=1).all()).CompID.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  39,  40,  41,  42,  43,  44,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 114, 115, 116, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 167, 169,
       170, 171, 172, 174, 176, 177, 179, 185, 186, 192, 193, 196, 197,
       199, 210, 213, 216, 217, 221, 229, 255, 351, 356, 365, 416, 485,
       804], dtype=int64)

In [271]:
df[df.CompID == 485]

,CompID,Name,Sex,DOB,BDW,Class,Division,SQ1,SQ2,SQ3,BestSQ,BP1,BP2,BP3,BestBP,DL1,DL2,DL3,BestDL,Total,Wilks,Place
Index,,,,,,,,,,,,,,,,,,,,,,
9293,485,Smoot Priscilla,F,1979.0,46.15,<NA>,<NA>,85.0,90.0,90.0,NaN,50.0,55.0,57.5,NaN,115.0,122.5,125.0,NaN,270.0,367.89,6.0
9294,485,Craven Elizabeth,F,1975.0,51.40,<NA>,<NA>,127.5,132.5,137.5,NaN,70.0,75.0,75.0,NaN,135.0,145.0,145.0,NaN,342.5,430.80,6.0
9295,485,Cauchi Kerri-Anne,F,1996.0,50.95,<NA>,<NA>,80.0,85.0,87.5,NaN,42.5,45.0,47.5,NaN,112.5,120.0,122.5,NaN,252.5,319.75,3.0
9296,485,Dayes Jordan,F,1992.0,50.75,<NA>,<NA>,97.5,105.0,105.0,NaN,40.0,42.5,45.0,NaN,105.0,115.0,115.0,NaN,247.5,314.36,7.0
9297,485,Newton Alix,F,1986.0,55.70,<NA>,<NA>,120.0,125.0,127.5,NaN,75.0,77.5,80.0,NaN,132.5,132.5,137.5,NaN,335.0,395.84,9.0
9298,485,Navarro Tamara,F,1992.0,56.75,<NA>,<NA>,110.0,117.5,117.5,NaN,47.5,50.0,52.5,NaN,117.5,122.5,132.5,NaN,282.5,328.94,5.0
9299,485,Sorrenson Catherine,F,1972.0,56.35,<NA>,<NA>,115.0,122.5,127.5,NaN,75.0,80.0,85.0,NaN,127.5,140.0,142.5,NaN,335.0,392.25,2.0
9300,485,Clarke Kelli,F,1980.0,62.35,<NA>,<NA>,145.0,145.0,152.5,NaN,67.5,72.5,75.0,NaN,150.0,162.5,170.0,NaN,387.5,419.45,7.0
9301,485,Symon Ariel,F,1992.0,60.95,<NA>,<NA>,117.5,125.0,127.5,NaN,70.0,77.5,77.5,NaN,125.0,130.0,135.0,NaN,330.0,363.46,4.0
